In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Adding Rafikamal To Path


pip install

In [0]:
!pip install python-docx
!pip install pyLDAvis
!pip install tabulate

Add RafiKamal To path

In [0]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/Library')

In [0]:
!ls drive/My\ Drive/Colab\ Notebooks/Library

In [0]:
from rafikamal2014 import parser

In [0]:
import gensim
import pandas as pd
import docx
import string
import numpy as np
import pyLDAvis.gensim
from gensim.test.utils import datapath
from tabulate import tabulate
from gensim.models.coherencemodel import CoherenceModel

import files

In [0]:
FILE_NAME = 'drive/My Drive/Colab Notebooks/Datasets/70k_bangla_newspaper.csv'
STOP_WORD_LIST_DIR = 'drive/My Drive/Colab Notebooks/Stopwords/StopWords.docx'

Boost Word in Eta Function


In [0]:
def boost_words_in_eta(eta, dictionary, boost_words):
    for i, word_list in enumerate(boost_words):
        boost_word_id_list = []
        print("{} is in {}".format(i, word_list))


        uniq, count = np.unique(eta, return_counts=True)
        print(dict(zip(uniq, count)))

        #print([list(dictionary.keys())[list(dictionary.values()).index(each_word)] for each_word in word_list])

        for each_word in word_list:

            try:
                word_id = list(dictionary.keys())[list(dictionary.values()).index(each_word)]
                print(word_id)
                boost_word_id_list.append(word_id)

            except ValueError:
                print("{} Not found in Dictionary ".format(each_word))

        for word in boost_word_id_list:
            eta[i][word] = 0.75

    uniq, count = np.unique(eta, return_counts=True)
    print(dict(zip(uniq, count)))

    return eta

**Test Unseen Document**

In [0]:
def unseen_doc_check(docs, model, dictionary):

  pass

Prepare Bag of Words Function

In [0]:
def prepare_bag_of_words(processed_docs, dictionary):
    return [dictionary.doc2bow(doc) for doc in processed_docs]

Read doc as Pandas DF Function

In [0]:
def read_doc_as_pandasDF(filename):

    data = pd.read_csv(filename, error_bad_lines=False)
    data_text = data[['content']]

    data_text['index'] = data_text.index
    documents = data_text

    return documents

Load Stop Word Function

In [0]:
def load_stop_word():

    global STOP_WORD_LIST_DIR

    stop_directory = STOP_WORD_LIST_DIR

    doc = docx.Document(stop_directory)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)

    bengali_stop_words = fullText[0].split()
    bengali_stop_words = frozenset(bengali_stop_words)
    #print(bengali_stop_words)

    return bengali_stop_words

Preprocess Documents Function

In [0]:
def preprocess_documents(doc):

    preprocessed_list_of_docs = []
    stemmer = parser.RafiStemmer()

    stop_words = load_stop_word()
    preprocessed_docs = []

    doc_token = []

    if isinstance(doc, str):
        for token in punctuation_remover(doc).split():
            if token not in stop_words and len(token) >= 3:
                if len(stemmer.stem_word(token)) >= 2:
                    doc_token.append(stemmer.stem_word(token))


    return doc_token

Punctuation Remover FUnction


In [0]:
def punctuation_remover(text):
    BENGALI_PUNCTUATION = string.punctuation + "—।’‘"
    BENGALI_NUMERALS = "০১২৩৪৫৬৭৮৯"
    return text.translate(str.maketrans(' ', ' ', BENGALI_PUNCTUATION+BENGALI_NUMERALS))

Main Function

In [0]:
if __name__ == '__main__':

    np.random.seed(2019)

    NUM_TOPICS = 8

    pd_document = read_doc_as_pandasDF(FILE_NAME)
    smaller_documents = pd_document

    #smaller_documents = pd_document[:20]

    # processed_docs after stop word removal && length

    #print(smaller_documents['content'][2])
    processed_docs = smaller_documents['content'].map(preprocess_documents)

    print("Printing Processed Docs\n")
    print(processed_docs)

    print('\n Preparing and showing dictionary \n')

    # Adjust no_below and no_above
    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=.01, no_above=0.6, keep_n=100000)

    print(dictionary)

    bow_corpus = prepare_bag_of_words(processed_docs, dictionary)

    print(" bow corpus number 5 entry:", end=" ")
    print(bow_corpus[5])

    eta = np.ones((NUM_TOPICS, len(dictionary))) * 0.1
    print("Shape of eta is {}".format(eta.shape))
    print("type of dictionary {}".format(type(dictionary)))
    print(eta)


    lda_model_normal = gensim.models.LdaMulticore(bow_corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=2, workers=3)




Print LDA Topics


In [0]:
    print("LDA Model:")

    for idx in range(NUM_TOPICS):
            # Print the first 10 most representative topics
        print("Topic #%s:" % idx, lda_model_normal.print_topic(idx, 10))

    print("=" * 20)

**Load Test Data Set**

In [0]:
TEST_DATA_SET = "drive/My Drive/Colab Notebooks/TestDataSet/bd20.csv"

test_df = read_doc_as_pandasDF(TEST_DATA_SET)

In [0]:
for unseen_document in test_df['content']:

  print("-----------------------")
  bow_vector = dictionary.doc2bow(preprocess_documents(unseen_document))

  for index, score in sorted(lda_model_normal[bow_vector], key=lambda tup: -1*tup[1]):
      print("Score: {}\t Topic:{} {}".format(score,index, lda_model_normal.print_topic(index, 5)))

**Model Save Test**

In [0]:
temp_file = datapath("model")
lda_model_normal.save("/content/drive/My Drive/Colab Notebooks/Cluster/Saved_Knowledge/lda_8topics.model")

Dictionary Save 


In [0]:
dictionary.save("drive/My Drive/Colab Notebooks/Saved Dictionary/word_dictionary.dic")

load dictionary

In [0]:
dictionary = gensim.corpora.Dictionary.load("drive/My Drive/Colab Notebooks/Saved Dictionary/word_dictionary.dic")

In [0]:
print((dictionary))
print(dictionary.get(10))

Reopen Model
**bold text**

In [0]:
lda_model_normal = gensim.models.LdaModel.load("drive/My Drive/Colab Notebooks/Saved model/lda.model")

Save Model


In [0]:
model_save_name = 'LDA_model.pt'
path = F"gdrive/My Drive/Colab Notebooks/Saved model{model_save_name}"
model.load_state_dict(torch.load(path))

Save **Normal**

In [0]:
    vis_normal = pyLDAvis.gensim.prepare(lda_model_normal, bow_corpus, dictionary =lda_model_normal.id2word)

    pyLDAvis.save_html(vis_normal, 'lda_normal22.html')

**Coherence**

In [0]:
    # Compute Coherence Score using c_v
    coherence_model_lda = CoherenceModel(model=lda_model_normal, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score C_V: ', coherence_lda)

Clustering
